### Assumptions
1. Every professor will have a unique name
2. A student will belong to only one section and take classes with that section only

# Constants

In [1]:
import enum
class Days(enum.Enum):
    MONDAY=0
    TUESDAY=1
    WEDNESDAY=2
    THURSDAY=3
    FRIDAY=4

In [2]:
class StartTimeSlots(enum.Enum):
    t9_00=1
    t10_00=2
    t11_00=3
    t12_00=4
    t13_00=5
    t14_00=6
    t15_00=7
    t16_00=8

In [3]:
class Classrooms(enum.Enum):
    A301=0
    A302=1
    A303=2
    A305=3
    A310=4
    A311=5

In [4]:
POPULATION_SIZE=10

In [5]:
hard_constraints=[
    "No teacher can hold two classes at the same time",
"No section can listen for two classes at the same time",
"No classroom can receive two classes at the same time",
"No teacher can hold three consecutive classes",
"There will be no class before 8:30 am and after 5:00 pm",
"University will remain close as there will be no class on weekends (Sat, Sun)"
]

In [6]:
soft_constraints=[
    "There will be no class from 1-2 on Friday.",
    "No section can hold three consecutive classes",
    "A subject having multiple forms, such as lectures and labs, the preferred order is: lecture and lab",
    "One hour for faculty meeting in a week when there is no class except Friday’s prayer break."
]

# Classes

In [7]:
import random
class Gene:
    def __init__(self):
        self.day=random.choice(list(Days))
        self.startTimeSlot=random.choice(list(StartTimeSlots))
        self.classroom=random.choice(list(Classrooms))
        #self.day=Days.MONDAY
        #self.startTimeSlot=StartTimeSlots.t9_00
        #self.classroom=Classrooms.A301
        #print("("+str(self.day)+","+str(self.startTimeSlot)+","+str(self.classroom)+"), ")

In [8]:
class Chromosome:
    def __init__(self,slots):
        self.slots=slots #list of slots
        self.hardSatisfied=[False,False,False,False,False,False]
        self.softSatisfied=[False,False,False,False]
        
    #max possible fitness = 30*6+5+4=200
    def getFitness(self):
        fit=0
        for h in self.hardSatisfied:
            if h:
                fit+=30
        for s in self.softSatisfied:
            if s:
                fit+=5
        return fit

In [9]:
class Slot:
    def __init__(self,name,course,section):
        self.name=name
        self.course=course
        self.section=section
        self.duration=1 #in hours
        self.gene=None
        
    def __str__(self):
        return "name: "+self.name+", course: "+self.course+", section: "+self.section+", duration: "+self.duration

# Course-Instructor Allocation

In [10]:
instructors=["Ammad","Ibrar","Mishaal","Mesha","Zubaria"]
course=["AI","DS","CA","ITC","HCI"]
sections=["A","B","C","D","E"]
chromosomes=[]
for i in range(POPULATION_SIZE):
    listt=[]
    for k in range(len(instructors)):
        for j in range(len(sections)):
            p1=Slot(instructors[k],course=course[k],section=sections[j])
            print(instructors[k],course[k],sections[j])
            listt.append(p1)
    print()
    chromosomes.append(Chromosome(listt))

Ammad AI A
Ammad AI B
Ammad AI C
Ammad AI D
Ammad AI E
Ibrar DS A
Ibrar DS B
Ibrar DS C
Ibrar DS D
Ibrar DS E
Mishaal CA A
Mishaal CA B
Mishaal CA C
Mishaal CA D
Mishaal CA E
Mesha ITC A
Mesha ITC B
Mesha ITC C
Mesha ITC D
Mesha ITC E
Zubaria HCI A
Zubaria HCI B
Zubaria HCI C
Zubaria HCI D
Zubaria HCI E

Ammad AI A
Ammad AI B
Ammad AI C
Ammad AI D
Ammad AI E
Ibrar DS A
Ibrar DS B
Ibrar DS C
Ibrar DS D
Ibrar DS E
Mishaal CA A
Mishaal CA B
Mishaal CA C
Mishaal CA D
Mishaal CA E
Mesha ITC A
Mesha ITC B
Mesha ITC C
Mesha ITC D
Mesha ITC E
Zubaria HCI A
Zubaria HCI B
Zubaria HCI C
Zubaria HCI D
Zubaria HCI E

Ammad AI A
Ammad AI B
Ammad AI C
Ammad AI D
Ammad AI E
Ibrar DS A
Ibrar DS B
Ibrar DS C
Ibrar DS D
Ibrar DS E
Mishaal CA A
Mishaal CA B
Mishaal CA C
Mishaal CA D
Mishaal CA E
Mesha ITC A
Mesha ITC B
Mesha ITC C
Mesha ITC D
Mesha ITC E
Zubaria HCI A
Zubaria HCI B
Zubaria HCI C
Zubaria HCI D
Zubaria HCI E

Ammad AI A
Ammad AI B
Ammad AI C
Ammad AI D
Ammad AI E
Ibrar DS A
Ibrar DS B
Ibrar

In [11]:
chromosomes

# Time Table Generation using GA

In [12]:
def generateRandomGenes(): #generates random genes for each class slot
    for timetable in chromosomes: 
        for _class in timetable.slots:
            _class.gene=Gene()
        #print()

In [13]:
import numpy as np
import operator
def checkConstraintSatisfaction(chromosome): #checks if the constraints are being satisfied by a chromosome
    #check datatypye of chromosome is list of Slot
    
    #check hard constraint at index 0: No teacher can hold two classes at the same time
    chromosome.hardSatisfied[0]=True
    for instructor in instructors:
        allSlots=[slot for slot in chromosome.slots if slot.name == instructor]
        for slot in range(len(allSlots)):
            for slot2 in range(slot+1,len(allSlots)):
                if allSlots[slot].gene.day==allSlots[slot2].gene.day and allSlots[slot].gene.startTimeSlot==allSlots[slot2].gene.startTimeSlot:
                    print(allSlots[slot].name,allSlots[slot].course, allSlots[slot].section, allSlots[slot].gene.classroom, allSlots[slot].gene.day, allSlots[slot].gene.startTimeSlot)
                    print(allSlots[slot2].name,allSlots[slot2].course, allSlots[slot2].section, allSlots[slot2].gene.classroom, allSlots[slot2].gene.day, allSlots[slot2].gene.startTimeSlot)
                    print("----------No teacher can hold two classes at the same time")
                    chromosome.hardSatisfied[0]=False
                    
    # check hard constrainst at index 1: No section can listen for two classes at the same time
    chromosome.hardSatisfied[1]=True
    for section in sections:
        allSlots=[slot for slot in chromosome.slots if slot.section==section]
        for slot in range(len(allSlots)):
            for slot2 in range(slot+1,len(allSlots)):
                if allSlots[slot].gene.day==allSlots[slot2].gene.day and allSlots[slot].gene.startTimeSlot==allSlots[slot2].gene.startTimeSlot:
                    print(allSlots[slot].name,allSlots[slot].course, allSlots[slot].section, allSlots[slot].gene.classroom, allSlots[slot].gene.day, allSlots[slot].gene.startTimeSlot)
                    print(allSlots[slot2].name,allSlots[slot2].course, allSlots[slot2].section, allSlots[slot2].gene.classroom, allSlots[slot2].gene.day, allSlots[slot2].gene.startTimeSlot)            
                    print("----------No section can listen for two classes at the same time")
                    chromosome.hardSatisfied[1]=False
    
    #check hard constraint at index 2: No classroom can receive two classes at the same time
    chromosome.hardSatisfied[2]=True
    for classroom in Classrooms:
        allSlots=[slot for slot in chromosome.slots if slot.gene.classroom == classroom]
        for slot in range(len(allSlots)):
            for slot2 in range(slot+1,len(allSlots)):
                if allSlots[slot].gene.day==allSlots[slot2].gene.day and allSlots[slot].gene.startTimeSlot==allSlots[slot2].gene.startTimeSlot:
                    print(allSlots[slot].name,allSlots[slot].course, allSlots[slot].section, allSlots[slot].gene.classroom, allSlots[slot].gene.day, allSlots[slot].gene.startTimeSlot)
                    print(allSlots[slot2].name,allSlots[slot2].course, allSlots[slot2].section, allSlots[slot2].gene.classroom, allSlots[slot2].gene.day, allSlots[slot2].gene.startTimeSlot)
                    print("----------No classroom can receive two classes at the same time")
                    chromosome.hardSatisfied[2]=False
    
    #check hard constraint at index 3: No teacher can hold three consecutive classes
    chromosome.hardSatisfied[3]=True
    for instructor in instructors:
        for day in Days:
            allSlots=[slot for slot in chromosome.slots if slot.name == instructor and slot.gene.day==day]
            if len(allSlots)<=2:
                continue
            allSlots.sort(key=operator.attrgetter('gene.startTimeSlot.value'))
            for slot in range(len(allSlots)-2):
                if (allSlots[slot].gene.startTimeSlot.value+1==allSlots[slot+1].gene.startTimeSlot.value and allSlots[slot].gene.startTimeSlot.value+2==allSlots[slot+2].gene.startTimeSlot.value):
                    print(allSlots[slot].name,allSlots[slot].course, allSlots[slot].section, allSlots[slot].gene.classroom, allSlots[slot].gene.day, allSlots[slot].gene.startTimeSlot)
                    print("----------No teacher can hold three consecutive classes")
                    chromosome.hardSatisfied[3]=False
    
    #check hard constraint at index 4: There will be no class before 8:30 am and after 5:00 pm
    #already ensured when defining range for startTimeSlot
    chromosome.hardSatisfied[4]=True
    
    #check hard constraint at index 0: University will remain close as there will be no class on weekends (Sat, Sun)
    #already ensured when defining range for day
    chromosome.hardSatisfied[5]=True

In [14]:
generateRandomGenes()
for chromosome in chromosomes:
    checkConstraintSatisfaction(chromosome)
    print(chromosome.getFitness())
    print()

180

Mishaal CA C Classrooms.A302 Days.MONDAY StartTimeSlots.t11_00
Mishaal CA E Classrooms.A302 Days.MONDAY StartTimeSlots.t11_00
----------No teacher can hold two classes at the same time
Ammad AI C Classrooms.A305 Days.MONDAY StartTimeSlots.t16_00
Mesha ITC C Classrooms.A305 Days.MONDAY StartTimeSlots.t16_00
----------No section can listen for two classes at the same time
Ammad AI A Classrooms.A302 Days.MONDAY StartTimeSlots.t10_00
Ibrar DS D Classrooms.A302 Days.MONDAY StartTimeSlots.t10_00
----------No classroom can receive two classes at the same time
Mishaal CA C Classrooms.A302 Days.MONDAY StartTimeSlots.t11_00
Mishaal CA E Classrooms.A302 Days.MONDAY StartTimeSlots.t11_00
----------No classroom can receive two classes at the same time
Ammad AI C Classrooms.A305 Days.MONDAY StartTimeSlots.t16_00
Mesha ITC C Classrooms.A305 Days.MONDAY StartTimeSlots.t16_00
----------No classroom can receive two classes at the same time
90

Zubaria HCI B Classrooms.A303 Days.THURSDAY StartTimeSl